# Rede Neural

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# DEFINI VARIÁVEIS

In [2]:
def inicia_variaveis():
    
    dataHora = datetime.now()
    dataHora = dataHora.strftime("%d_%m_%Y_%H")

    global tempoInicialGeral, tempoFinalGeral, identificadas, baixo, alto, fator_imgs, altura_minima_ponto, altura_maxima_ponto
    global corretas, erradas, info, sensibilidade, mediaHistograma, LER_IMG, ESCREVE_IMG, distancia_pontos_raio_px, delta_distancia_lidar, reducao_caixa_fruta
    global TIPO_BASE, limiarLarguraPedunculo, limiarAlturaPedunculo, NOME_IMAGEM, NOME_JSON, NOME_TXT, METODO, min_samples,metodo_pontos_curva,altura_caixa_pedunculo, EXECUTA_VALIDACAO, QUANTIDADE_IMAGEM,POSICAO_PONTO, RAIO_CORTE, VERIFICACAO_PROFUNDIDADE_MANGA, QUANTIDADE_CORTE
    
    tempoInicialGeral = 0
    tempoFinalGeral = 0
    
    corretas =        []
    erradas =         []
    info =            []
    sensibilidade =   []
    mediaHistograma = []

    limiarAlturaPedunculo  = 0.3
    limiarLarguraPedunculo = 0.2

    baixo = [2, 0, 0]
    alto  = [31, 255, 255]

    distancia_pontos_raio_px = 2

    fator_imgs = 7.5
    
    altura_minima_ponto = 3
    altura_maxima_ponto = 4
    
    delta_distancia_lidar = 10
    
    reducao_caixa_fruta = 0.2
    
    altura_caixa_pedunculo = 10
    
    EXECUTA_VALIDACAO = True
    
    EXECUTA_VALIDACAO_GERAL = True
    
    min_samples = 4
    metodo_pontos_curva = "IsolationForest"
    #metodo_pontos_curva = "Outliners"
    
    TIPO_BASE = "3D"
        
    METODO = "DBSCAN"
    #METODO = "Kmeans"
    #METODO = "Heuristica"
    
    QUANTIDADE_CORTE = 1
    
    if(TIPO_BASE == "3D"):
    
        QUANTIDADE_CORTE = 3
    
    VERIFICACAO_PROFUNDIDADE_MANGA = True
    
    ##################2222222222222222222222222222222222DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD############################
    
    #PASTA = "VALID_CUT_POINT_2D/"
    
    #LER_IMG           = "VI2D_MANGO_LEAFLESS_SEM_FOLHA/"
    #NOME_IMAGEM       = "_MANGO_LEAFLESS"
    #QUANTIDADE_IMAGEM = 242
    #ESCREVE_IMG = "VALID_CUT_POINT_2D/RESULTADO_SEM_FOLHA/"
    
    #LER_IMG          = "VI2D_MANGO_LEAFY_BACKGROUND_COM_FOLHA/"
    #NOME_IMAGEM      = "_MANGO_LEAFY_BACKGROUND"
    #QUANTIDADE_IMAGEM = 201
    #ESCREVE_IMG = "VALID_CUT_POINT_2D/RESULTADO_COM_FOLHA/"
        
    #LER_IMG          = "VI2D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/"
    #NOME_IMAGEM      = "_STEM_OCCLUDED"
    #QUANTIDADE_IMAGEM = 166
    #ESCREVE_IMG = "VALID_CUT_POINT_2D/RESULTADO_PEDUNCULO_OCLUSO/"
    
    
    ##################3333333333333333333333333333333DDDDDDDDDDDDDDDDDDDDDDDDDDDDDD#################################
    
    PASTA = "VALID_CUIT_POINT_3D/"
    
    #LER_IMG           = PASTA + "VI3D_MANGO_LEAFLESS_SEM_FOLHA/"
    #NOME_IMAGEM       = "_MANGO_LEAFLESS"
    #QUANTIDADE_IMAGEM = 153
    
    #LER_IMG          = PASTA + "VI3D_MANGO_LEAFY_BACKGROUND_COM_FOLHA/"
    #NOME_IMAGEM      = "_MANGO_LEAFY_BACKGROUND"
    #QUANTIDADE_IMAGEM = 166
    
    LER_IMG          = PASTA + "VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/"
    NOME_IMAGEM      = "_STEM_OCCLUDED"   
    QUANTIDADE_IMAGEM = 76
    
    ARQUIVOS_LOG = PASTA + "RESULTADOS/" + NOME_IMAGEM[1:] + "_" + METODO + "_" + dataHora
    ARQUIVOS_LOG = ARQUIVOS_LOG.upper()
    ESCREVE_IMG = ARQUIVOS_LOG + "/"
    
    NOME_JSON   = "_MANGO_MAP_JSON"
    
    NOME_TXT = ARQUIVOS_LOG
    
    if(not os.path.exists(ESCREVE_IMG)):
        
        os.mkdir(ESCREVE_IMG)

# FUNÇÃO PRINCIPAL

In [3]:
import cv2 
import numpy as np
import time
import traceback
import math
import json
import os
from datetime import datetime

import PC_SVCF as pc
import Lidar as lidar
import Valida_SVCF as vl

id_imagem = 1
QUANTIDADE_IMAGEM = 0

inicia_variaveis()

print("Executando...")
print("Log..." + NOME_TXT)
arquivo = open(NOME_TXT + ".csv", "w")

identificadas = 0
naoIdentificadas = 0

quantidadeManga = 0

contadores = {
    1: 0,
    2: 0,
    3: 0
}

corretas_2d = 0

if(TIPO_BASE == "2D"):
    
    arquivo.write("IMAGEM,ID_MANGA,SCORES,HUE,%_MANGA,%_PONTO\n")
    
    tamanhoLetraTexto = 0.6
    fatorLetra = 20
    
else:
    
    arquivo.write("IMAGEM,ID_MANGA,SCORES,FATOR_CM_PX,DISTÂNCIA_MANGA,HUE,%_MANGA,%_PEDÚNCULO,DISTÂNCIA_PONTO,DIFERENÇA_HORIZONTAL_P,DIFERENÇA_VERTICAL_P,DIFERENÇA_VERTICAL_C,PONTO_1,PONTO_2,PONTO_3\n")
    
    tamanhoLetraTexto = 1.0
    fatorLetra = 35

tempoInicialGeral = time.time()

pontosJson = ""
fator_cm_px = ""

media_final = [ ]

while(id_imagem <= QUANTIDADE_IMAGEM):
    
    inicia_variaveis()

    print("############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################\n")
    
    caminho_imagem = LER_IMG + str(id_imagem) + NOME_IMAGEM +  ".jpg"
    print(f"Caminho da imagem: {caminho_imagem}")
    
    imagem_original = cv2.imread(LER_IMG + str(id_imagem) + NOME_IMAGEM + ".jpg")
        
    caminho_json = LER_IMG + str(id_imagem) + NOME_JSON + ".json"
    print(f"Caminho do json:   {caminho_json}\n")
    
    if(TIPO_BASE == "3D"):

        pontosJson = lidar.organizaJSON(caminho_json)
        
    #Copia a imagem original
    imagem_original_copia = np.array(imagem_original)
    
    imagem_detectada = imagem_original_copia.copy() #USADA DURANTE A EXECUÇÃO DO CÓDIGO
    imagem_pedunculo = imagem_original_copia.copy() #USADA PARA A ÁREA DO PEDÚNCULO
    
    #Copia a imagem na versão reduzida
    imagem_original_reduzida_aux = cv2.resize(imagem_original, (192, 256))
    imagem_original_reduzida = np.array(imagem_original_reduzida_aux.astype(np.uint8))
    
    ################################ CONFIGURAÇÕES DA REDE NEURAL ################################
    
    category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')
        
    input_tensor = tf.convert_to_tensor(np.expand_dims(imagem_original_copia, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor) 

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    
    detection_threshold = 0.8

    viz_utils.visualize_boxes_and_labels_on_image_array(
                imagem_detectada,
                detections['detection_boxes'],
                detections['detection_classes'] + label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                skip_scores= True,
                skip_labels= True,
                skip_boxes= True,
                agnostic_mode=False)
  
    scores = list(filter(lambda x: x> detection_threshold, 
    detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
        
    width =  imagem_detectada.shape[1]
    height = imagem_detectada.shape[0]
        
    sensibilidade.append(scores)
    
    quantidade_fruta_identificadas = len(scores)
    
    ################################ INICIO DO PROCESSO DE LOCALIZAÇÃO DO PONTO DE CORTE ################################
    
    if(quantidade_fruta_identificadas > 0):
                
        print(f"Imagem: {id_imagem} - Quantidade de fruta identificada: {quantidade_fruta_identificadas}\n")
                
        try:
            
            identificadas += 1
            
            posicaoTextoY = fatorLetra
            
            for id_manga_localizada, box in enumerate(boxes):
                
                arquivo.write(f"{str(id_imagem)},")
                
                quantidadeManga += 1
                                
                print(f"Manga localizada: {id_manga_localizada}")
                
                if(id_manga_localizada < 2):
                    
                    arquivo.write(f"{str(id_manga_localizada)},")
                    arquivo.write(f"{str(round(scores[id_manga_localizada], 4))},")
                                                            
                    ################################# ÁREA DA MANGA #################################

                    roi = box * [height, width, height, width]
                    
                    xt_ia_manga = roi[1]
                    yt_ia_manga = roi[0]
                    
                    xb_ia_manga = roi[3]
                    yb_ia_manga = roi[2]

                    xt_ia_manga = int(xt_ia_manga)
                    xb_ia_manga = int(xb_ia_manga)
                    yt_ia_manga = int(yt_ia_manga)
                    yb_ia_manga = int(yb_ia_manga)
                    
                    tamanho_caixa_manga = abs(yt_ia_manga - yb_ia_manga)
                    
                    cv2.putText(imagem_detectada, str(id_manga_localizada), (xt_ia_manga, yt_ia_manga - 5), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)
                                        
                    centroCaixaFrutaX = int((xt_ia_manga + xb_ia_manga) / 2)
                    centroCaixaFrutaY = int((yt_ia_manga + yb_ia_manga) / 2)
                    
                    centroCaixaFrutaXLidar = int(centroCaixaFrutaX / fator_imgs)
                    centroCaixaFrutaYLidar = int(centroCaixaFrutaY / fator_imgs)
                    
                    media_imagem = [ ]
                    
                    if(TIPO_BASE == "3D"):
                    
                        #Função para encontrar o fator cm_px referente a manga
                        fator_cm_px = pc.obter_cm_px(4, 2, centroCaixaFrutaXLidar, centroCaixaFrutaYLidar, pontosJson)
                                                
                        arquivo.write(f"{str(round(fator_cm_px, 4))},")

                        print(f"Fator cm_px: {fator_cm_px:.5f}")
                                            
                        refencia_altura_caixa_pedunculo = 4.5 / fator_cm_px * fator_imgs
                        
                        limiarAlturaPedunculo = round(refencia_altura_caixa_pedunculo / tamanho_caixa_manga, 2)
                                                
                        distanciaFruta = lidar.measureDistanceOnePoint((centroCaixaFrutaXLidar, centroCaixaFrutaYLidar), pontosJson)
                        
                        print(f"Distancia Manga: {distanciaFruta:.2f} cm")
                        
                        arquivo.write(f"{str(round(distanciaFruta, 2))},")
                            
                        if(distanciaFruta > 40.0):
                            
                            VERIFICACAO_PROFUNDIDADE_MANGA = False
                            
                            arquivo.write("\n")
                            
                            quantidadeManga -= 1
                            
                        else:

                            cv2.putText(imagem_detectada, "Distancia Manga - " + str(id_manga_localizada) + ": "+ str(round(distanciaFruta, 2)) + " cm", (20, posicaoTextoY),cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                            posicaoTextoY += fatorLetra
                            
                    if(VERIFICACAO_PROFUNDIDADE_MANGA):
                        
                        #Faz um corte na área da manga
                        areaManga = imagem_detectada[yt_ia_manga:yb_ia_manga, xt_ia_manga:xb_ia_manga]

                        #Circulo no centro da área da manga
                        cv2.circle(imagem_detectada, (int(centroCaixaFrutaX), int(centroCaixaFrutaY)), 5, (255, 0, 90), -1)

                        #Desenha um retangulo na área da manga
                        cv2.rectangle(imagem_detectada, (xt_ia_manga, yt_ia_manga), (xb_ia_manga, yb_ia_manga), (255,0,0), 2)


                        ################################# ÁREA DO PEDÚNCULO ################################# limiarAlturaPedunculo

                        #Chama a função que prevê a área do pedúnculo
                        areaPedunculo = pc.preveAreaPedunculo(xt_ia_manga, yt_ia_manga, xb_ia_manga, yb_ia_manga, limiarLarguraPedunculo, limiarAlturaPedunculo, -altura_caixa_pedunculo)

                        #x1                y1                 x2                 y2
                        xt_area_pedunculo, yt_area_pedunculo, xb_area_pedunculo, yb_area_pedunculo = areaPedunculo[0], areaPedunculo[1], areaPedunculo[2], areaPedunculo[3]

                        #Desenha um retangulo na área do pedúnculo        
                        cv2.rectangle(imagem_detectada, (xt_area_pedunculo, yt_area_pedunculo), (xb_area_pedunculo, yb_area_pedunculo), (0, 0, 255), 2)

                        #Coordenadas da área da manga
                        TopLeftX = xt_area_pedunculo
                        TopLeftY = yb_area_pedunculo

                        #Faz o corte na área prevista do pedúnculo
                        corteAreaPedunculo = imagem_pedunculo[yb_area_pedunculo:yt_area_pedunculo, xt_area_pedunculo:xb_area_pedunculo]                    

                        if(len(corteAreaPedunculo) < 1):

                            print("Área do pedúnculo muito pequena")

                        else:

                            #Circulo no centro do topo da área da fruta
                            cv2.circle(imagem_detectada, (int(centroCaixaFrutaX), int(yt_ia_manga)), 5, (0, 0, 255), -1)

                            if(TIPO_BASE == "3D"):

                                #Distancia do centro do topo da caixa do fruta
                                distanciaTopoCaixaFruto = lidar.measureDistanceOnePoint((centroCaixaFrutaXLidar, int(yt_ia_manga / fator_imgs)), pontosJson)

                                print(f"Distancia Topo Caixa: {distanciaTopoCaixaFruto:.2f} cm")

                            #Função que encontra o ponto de corte final de acordo com os parâmetros
                            ponto_final = pc.localiza_ponto_final(str(id_imagem),
                                                    str(id_manga_localizada),
                                                    corteAreaPedunculo, 
                                                    baixo, 
                                                    alto, 
                                                    METODO, 
                                                    ESCREVE_IMG, 
                                                    True, 
                                                    None, 
                                                    7, 
                                                    min_samples, 
                                                    TIPO_BASE, 
                                                    metodo_pontos_curva, 
                                                    TopLeftX, 
                                                    TopLeftY, 
                                                    pontosJson, 
                                                    altura_minima_ponto, 
                                                    (centroCaixaFrutaX, centroCaixaFrutaY), 
                                                    (centroCaixaFrutaX, yt_ia_manga))

                            pontoX, pontoY, valorHue, distanciaPonto, distanciaProfundidadePonto = ponto_final[0], ponto_final[1], ponto_final[2], ponto_final[3], ponto_final[4]

                            arquivo.write(f"{str(valorHue)},")

                            if(EXECUTA_VALIDACAO):
                                
                                incremento_corte = 1
                                
                                while(incremento_corte <= QUANTIDADE_CORTE):

                                    #Função para validar o sistema
                                    validacao = vl.valida_svcf(imagem_detectada, 
                                                   None,
                                                   id_imagem,
                                                   id_manga_localizada,
                                                   TIPO_BASE,
                                                   (xt_ia_manga, yt_ia_manga, xb_ia_manga, yb_ia_manga),
                                                   (xt_area_pedunculo, yt_area_pedunculo, xb_area_pedunculo, yb_area_pedunculo),
                                                   caminho_json, 
                                                   fator_cm_px,
                                                   (pontoX, pontoY),
                                                   altura_minima_ponto,
                                                   distanciaPonto, 
                                                   0, 
                                                   incremento_corte)

                                    por_manga, por_pedunculo, por_ponto, distancia_hor, distancia_vert, distanciaPontoConhecido = validacao[0], validacao[1], validacao[2], validacao[3], validacao[4], validacao[5]

                                    if(incremento_corte == 1):
                                    
                                        arquivo.write(f"{str(por_manga)},")

                                        print(f"Porcentagem Manga:     {por_manga} %")
                                        cv2.putText(imagem_detectada, "Precisao Manga: "+ str(por_manga) + " %", (20, posicaoTextoY),cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                        posicaoTextoY += fatorLetra

                                        if(TIPO_BASE == "3D"):

                                            print(f"Porcentagem Pedunculo: {por_pedunculo} %")
                                            print(f"Distancia Horizontal:  {distancia_hor} cm")
                                            print(f"Distancia Vertical:    {distancia_vert} cm")
                                            print(f"Ponto <--> Distancia Conhecida: {distanciaPontoConhecido} cm")

                                            arquivo.write(f"{str(por_pedunculo)},")
                                            arquivo.write(f"{str(round(distanciaPonto, 2))},")
                                            arquivo.write(f"{str(distancia_hor)},")
                                            arquivo.write(f"{str(distancia_vert)},")

                                            cv2.putText(imagem_detectada,"Precisao Pedunculo: " + str(por_pedunculo) + " %", (20, posicaoTextoY),cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                            posicaoTextoY += fatorLetra

                                            cv2.putText(imagem_detectada, "Distancia Hor: " + str(distancia_hor) + " cm", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                            posicaoTextoY += fatorLetra

                                            cv2.putText(imagem_detectada, "Distancia Ver: " + str(distancia_vert) + " cm", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                            posicaoTextoY += fatorLetra

                                            cv2.putText(imagem_detectada, "Altura Ponto IA: " + str(distanciaPonto) + " cm", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                            posicaoTextoY += fatorLetra

                                            cv2.putText(imagem_detectada, "Altura Ponto Label: " + str(distanciaPontoConhecido) + " cm", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                            posicaoTextoY += fatorLetra

                                            cv2.putText(imagem_detectada, "Depht Ponto: " + str(distanciaProfundidadePonto) + " cm", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                            posicaoTextoY += fatorLetra

                                            arquivo.write(f"{str(round(abs(distanciaPontoConhecido - distanciaPonto), 2))},")

                                            #Salva imagem representativa da calibração
                                            cv2.imwrite(ESCREVE_IMG + str(id_imagem) + "_" + str(id_manga_localizada) + "_a_calibracao.png", imagem_original_reduzida)

                                    if(TIPO_BASE == "3D"):

                                        print(f"Porcentagem Ponto {incremento_corte}_cm:     {por_ponto}")
                                        cv2.putText(imagem_detectada, "Precisao Ponto " + str(incremento_corte) + "_cm: " + str(por_ponto), (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                        if(por_ponto == 1):
                                            
                                            if incremento_corte in contadores:
                                                
                                                contadores[incremento_corte] = str(int(contadores[incremento_corte]) + 1)
                                    
                                    else:

                                        print(f"Porcentagem Ponto:     {por_ponto}")
                                        cv2.putText(imagem_detectada, "Precisao Ponto: " + str(por_ponto), (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)
                                    
                                        if(por_ponto == 1):
                                            
                                            corretas_2d += 1
                                    
                                    cv2.circle(imagem_detectada, (pontoX, pontoY), 4, (255, 0, 255), -2)
                                    cv2.line(imagem_detectada, (pontoX, pontoY), (centroCaixaFrutaX, yt_ia_manga), (255, 0, 255), 2)
                                    cv2.circle(imagem_detectada, (pontoX, pontoY), 4, (255, 0, 0), 2)
                                    
                                    posicaoTextoY += fatorLetra
                                        
                                    arquivo.write(f"{str(por_ponto)},")
                                    
                                    #Salva imagem da área do pedúnculo RGB
                                    cv2.imwrite(ESCREVE_IMG + str(id_imagem) + "_" + str(id_manga_localizada) + "_b_area_pedunculo.jpg", corteAreaPedunculo)
                                    
                                    incremento_corte += 1

                            cv2.imwrite(ESCREVE_IMG + str(id_imagem) + "_" + str(id_manga_localizada) + "_j_ponto_final.jpg", imagem_detectada)
                            arquivo.write("\n")
                
            print("-*-*-*-*-*-*-*-*-*-*-*")
                        
            id_imagem = id_imagem + 1
                  
        except:

            print(f"{id_imagem} - Erro desconhecido\n") 
            
            traceback.print_exc()

            id_imagem = id_imagem + 1
            
    else:
        
        print("\n{} - Fruta nao identificada\n".format(id_imagem))
                
        naoIdentificadas += 1
        
        if(TIPO_BASE == "2D"):
    
            arquivo.write(f"{str(id_imagem)},-*-,-*-,-*-,-*-,-*-\n")
    
        else:
    
            arquivo.write(f"{str(id_imagem)},-*-,-*-,-*-,-*-,-*-,-*-,-*-,-*-,-*-,-*-,-*-,-*-\n")
    
        
        id_imagem = id_imagem + 1
        
tempoFinalGeral = time.time()
arquivo.write(f"\nTOTAL DE IMAGEM: {QUANTIDADE_IMAGEM}\n")
arquivo.write(f"Identificadas: {identificadas}\n")
arquivo.write(f"Não Identificadas: {naoIdentificadas}\n")

arquivo.close()
print("\n\nFinalizando...")

Executando...
Log...VALID_CUIT_POINT_3D/RESULTADOS/STEM_OCCLUDED_DBSCAN_25_03_2024_17
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho da imagem: VALID_CUIT_POINT_3D/VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/1_STEM_OCCLUDED.jpg
Caminho do json:   VALID_CUIT_POINT_3D/VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/1_MANGO_MAP_JSON.json

Imagem: 1 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.11576
Distancia Manga: 19.19 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 21.50 cm
Função: --> Segmenta Área Pedúnculo
Função: --> Método DBSCAN
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 25
Quantidade de Pontos Candidatos: 229
Quantidade de cluster encontrado: 4
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 0
Ponto no centro do area do pedúnculo
Função: --> Remove Pontos Fora da Curva
Função: --> Método IsolationForest
Quantidade d

Imagem: 7 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.13978
Distancia Manga: 22.45 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 23.62 cm
Função: --> Segmenta Área Pedúnculo
Função: --> Método DBSCAN
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 17
Quantidade de Pontos Candidatos: 477
Quantidade de cluster encontrado: 4
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 2
Função: --> Remove Pontos Fora da Curva
Função: --> Método IsolationForest
Quantidade de pontos selecionados: 2
Função: --> Seleciona ponto final
A distância do ponto selecionado para caixa IA: 3.37 cm
Distancia profundidade ponto selecionado: 23.67 cm
Porcentagem Manga:     100.0 %
Porcentagem Pedunculo: 39.47 %
Distancia Horizontal:  0.15 cm
Distancia Vertical:    0.47 cm
Ponto <--> Distancia Conhecida: 2.39 cm
Porcentagem Ponto 1_cm:     1
Porcentagem Ponto 2_cm:     1
P

Imagem: 14 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.13310
Distancia Manga: 22.27 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 22.89 cm
Função: --> Segmenta Área Pedúnculo
Função: --> Método DBSCAN
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 29
Quantidade de Pontos Candidatos: 513
Quantidade de cluster encontrado: 2
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 0
Ponto no centro do area do pedúnculo
Função: --> Remove Pontos Fora da Curva
Função: --> Método IsolationForest
Quantidade de pontos selecionados: 1
Função: --> Seleciona ponto final
A distância do ponto selecionado para caixa IA: 3 cm
Distancia profundidade ponto selecionado: 28.78 cm
Porcentagem Manga:     100.0 %
Porcentagem Pedunculo: 100.0 %
Distancia Horizontal:  0.35 cm
Distancia Vertical:    0.8 cm
Ponto <--> Distancia Conhecida: 2.23 cm
Porcentagem Ponto 1_cm:    

Quantidade de Pontos Candidatos: 110
Quantidade de cluster encontrado: 3
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 1
Função: --> Remove Pontos Fora da Curva
Função: --> Método IsolationForest
Quantidade de pontos selecionados: 1
Função: --> Seleciona ponto final
A distância do ponto selecionado para caixa IA: 3.59 cm
Distancia profundidade ponto selecionado: 55.22 cm
Porcentagem Manga:     95.15 %
Porcentagem Pedunculo: 23.26 %
Distancia Horizontal:  1.82 cm
Distancia Vertical:    1.1 cm
Ponto <--> Distancia Conhecida: 4.28 cm
Porcentagem Ponto 1_cm:     0
Porcentagem Ponto 2_cm:     0
Porcentagem Ponto 3_cm:     0
-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho da imagem: VALID_CUIT_POINT_3D/VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/21_STEM_OCCLUDED.jpg
Caminho do json:   VALID_CUIT_POINT_3D/VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/21_MANGO_MAP_JSON.json

Imagem: 21 - Quantidade de fruta iden

Quantidade de pontos selecionados: 2
Função: --> Seleciona ponto final
A distância do ponto selecionado para caixa IA: 3.48 cm
Distancia profundidade ponto selecionado: 47.07 cm
Porcentagem Manga:     99.54 %
Porcentagem Pedunculo: 10.34 %
Distancia Horizontal:  1.05 cm
Distancia Vertical:    0.11 cm
Ponto <--> Distancia Conhecida: 3.11 cm
Porcentagem Ponto 1_cm:     0
Porcentagem Ponto 2_cm:     0
Porcentagem Ponto 3_cm:     0
-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho da imagem: VALID_CUIT_POINT_3D/VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/27_STEM_OCCLUDED.jpg
Caminho do json:   VALID_CUIT_POINT_3D/VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/27_MANGO_MAP_JSON.json

Imagem: 27 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.16568
Distancia Manga: 27.10 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 31.64 cm
Função: --> Segmenta Área Pedúnculo
Função: --> Método DBSCAN
Funç

...

Imagem: 46 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.17084
Distancia Manga: 28.64 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 34.55 cm
Função: --> Segmenta Área Pedúnculo
Função: --> Método DBSCAN
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 23
Quantidade de Pontos Candidatos: 97
Quantidade de cluster encontrado: 3
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 1
Função: --> Remove Pontos Fora da Curva
Função: --> Método IsolationForest
Quantidade de pontos selecionados: 1
Função: --> Seleciona ponto final
A distância do ponto selecionado para caixa IA: 3.43 cm
Distancia profundidade ponto selecionado: 29.64 cm
Porcentagem Manga:     88.45 %
Porcentagem Pedunculo: 100.0 %
Distancia Horizontal:  0.09 cm
Distancia Vertical:    0.07 cm
Ponto <--> Distancia Conhecida: 3.08 cm
Porcentagem Ponto 1_cm:     1
Porcentagem Ponto 2_cm:     1
P

Quantidade de Pontos Candidatos: 475
Quantidade de cluster encontrado: 7
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 2
Função: --> Remove Pontos Fora da Curva
Função: --> Método IsolationForest
Quantidade de pontos selecionados: 2
Função: --> Seleciona ponto final
A distância do ponto selecionado para caixa IA: 3.72 cm
Distancia profundidade ponto selecionado: 39.48 cm
Porcentagem Manga:     98.64 %
Porcentagem Pedunculo: 100.0 %
Distancia Horizontal:  0.26 cm
Distancia Vertical:    0.29 cm
Ponto <--> Distancia Conhecida: 3.35 cm
Porcentagem Ponto 1_cm:     1
Porcentagem Ponto 2_cm:     1
Porcentagem Ponto 3_cm:     1
-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho da imagem: VALID_CUIT_POINT_3D/VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/53_STEM_OCCLUDED.jpg
Caminho do json:   VALID_CUIT_POINT_3D/VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/53_MANGO_MAP_JSON.json

Imagem: 53 - Quantidade de fruta ide

-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho da imagem: VALID_CUIT_POINT_3D/VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/59_STEM_OCCLUDED.jpg
Caminho do json:   VALID_CUIT_POINT_3D/VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/59_MANGO_MAP_JSON.json

Imagem: 59 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.11442
Distancia Manga: 19.62 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 20.08 cm
Função: --> Segmenta Área Pedúnculo
Função: --> Método DBSCAN
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 6
Quantidade de Pontos Candidatos: 259
Quantidade de cluster encontrado: 9
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 5
Função: --> Remove Pontos Fora da Curva
Função: --> Método IsolationForest
Quantidade de pontos selecionados: 4
Função: --> Seleciona ponto final
A distância do ponto selecionado para c

-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho da imagem: VALID_CUIT_POINT_3D/VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/66_STEM_OCCLUDED.jpg
Caminho do json:   VALID_CUIT_POINT_3D/VI3D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/66_MANGO_MAP_JSON.json

Imagem: 66 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.15633
Distancia Manga: 26.76 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 26.00 cm
Função: --> Segmenta Área Pedúnculo
Função: --> Método DBSCAN
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 29
Quantidade de Pontos Candidatos: 1332
Quantidade de cluster encontrado: 10
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 4
Função: --> Remove Pontos Fora da Curva
Função: --> Método IsolationForest
Quantidade de pontos selecionados: 4
Função: --> Seleciona ponto final
A distância do ponto selecionado par

Imagem: 72 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.11799
Distancia Manga: 19.38 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 32.06 cm
Função: --> Segmenta Área Pedúnculo
Função: --> Método DBSCAN
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 11
Quantidade de Pontos Candidatos: 105
Quantidade de cluster encontrado: 4
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 0
Ponto no centro do area do pedúnculo
Função: --> Remove Pontos Fora da Curva
Função: --> Método IsolationForest
Quantidade de pontos selecionados: 1
Função: --> Seleciona ponto final
A distância do ponto selecionado para caixa IA: 3 cm
Distancia profundidade ponto selecionado: 18.82 cm
Porcentagem Manga:     99.67 %
Porcentagem Pedunculo: 46.08 %
Distancia Horizontal:  0.38 cm
Distancia Vertical:    0.05 cm
Ponto <--> Distancia Conhecida: 3.09 cm
Porcentagem Ponto 1_cm:   

In [5]:
print(f"Método utilizado: {METODO}")
print(f"Base: {TIPO_BASE}")
print(f"\nImagens identificadas: {identificadas}")
print(f"Imagens não identificadas {naoIdentificadas}")
print("\nResultados: ")
print(f"Quantidade de manga: {quantidadeManga}")

if(TIPO_BASE == "3D"):

    for chave, valor in contadores.items():

        valor = int(valor)

        print(f"-->>Corte de {chave}cm")
        print(f"    Ponto correto: {valor}")
        print(f"    Porcentagem correto: {round((valor / quantidadeManga) * 100, 2)}%")
        print(f"    Ponto errado: {quantidadeManga - valor}")
        print(f"    Porcentagem errado: {round(((quantidadeManga - valor) / quantidadeManga) * 100, 2)}%\n")
        
else:
    
    print(f"Ponto correto: {corretas_2d}")
    print(f"Porcentagem correto: {round((quantidadeManga / corretas_2d) * 100, 2)}%")
    print(f"Ponto errado: {quantidadeManga - corretas_2d}")
    print(f"Porcentagem errado: {round(((quantidadeManga - corretas_2d) / quantidadeManga) * 100, 2)}%\n")

Método utilizado: DBSCAN
Base: 3D

Imagens identificadas: 73
Imagens não identificadas 3

Resultados: 
Quantidade de manga: 73
-->>Corte de 1cm
    Ponto correto: 55
    Porcentagem correto: 75.34%
    Ponto errado: 18
    Porcentagem errado: 24.66%

-->>Corte de 2cm
    Ponto correto: 64
    Porcentagem correto: 87.67%
    Ponto errado: 9
    Porcentagem errado: 12.33%

-->>Corte de 3cm
    Ponto correto: 65
    Porcentagem correto: 89.04%
    Ponto errado: 8
    Porcentagem errado: 10.96%

